# Data Preprocessing V2 

In [22]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

# PART A: Functions used in preprocessing

## A1. Overall helper functions

In [23]:
def identify_feats(df):
    # Categorical features
    cat_feats = df.select_dtypes(['object','bool']).columns.to_list()
    # Numerical features
    num_feats = df.select_dtypes(['int', 'float']).columns.to_list()
    print("categorical: "+ str(df[cat_feats].shape[1]))
    print("numerical: "+ str(df[num_feats].shape[1]))
    print("df dims:" + str(df.shape))
    return(cat_feats,num_feats)

In [24]:
def cols_with_na(df,threshold=0):
    count = 0
    for col in df.columns.to_list():
        if(df[col].isna().sum()/len(df))>threshold:
            count += 1
            print(str(col) + "   " + str(df[col].isna().sum()) + "   " + str(round(df[col].isna().sum()/len(df)*100,2))) 
    print('Total columns with NA values above threshold ' + str(threshold*100) + "%: " + str(count))

##  A2. Functions related to numeric features 
##### (and a few categorical in impute_missing_values)

In [25]:
#Special Case Imputations 
#Five numeric columns have missing values: LotFrontage (353), MasVnrArea (11), BsmtFullBath(1),  
#BsmtHalfBath(1), GarageYrBullt(92)
def impute_missing_vals(df):
    #1. Impute LotFrontage missing values with linear regression coefficients 
    # AA: LotFrontage imputed as (coefficient from dict) * sqrt(LotArea)
    LotFrontage_dict = {'1Fam':0.7139, 'TwnhsE':0.5849, 'Twnhs':0.5227, 'Duplex':0.7725, '2fmCon':0.6922}
    df.loc[df['LotFrontage'].isna(), 'LotFrontage'] = df.loc[df['LotFrontage'].isna(), :].apply(
        lambda x: LotFrontage_dict[x['BldgType']]*np.sqrt(x['LotArea']), axis=1)

    #2. All rows with MasVnrArea null values also have MasVnrType as null.\
    idx = df['MasVnrArea'].isna() & (df['MasVnrType'].isna())
    #Assume these properties do not have a veneer, so set MasVnrType as "None" and MasVnrArea as 0 
    df.loc[idx,'MasVnrArea'] = 0 
    df.loc[idx,'MasVnrType'] = "None" #motivated by the null value in test, is this data leakage?

    #3 & 4. BsmtFullBath & BsmtHalfBath nulls corresponds with No basement. Can impute with 0. 
    df.loc[df['BsmtFullBath'].isna() & (df['TotalBsmtSF']==0),'BsmtFullBath'] = 0
    df.loc[df['BsmtHalfBath'].isna() & (df['TotalBsmtSF']==0),'BsmtHalfBath'] = 0 

    #5. GarageYrBuilt - repalce missing year with YearBuilt for properties with non-zero garage area values  
    idx = df['GarageYrBlt'].isna() & (df['GarageArea']!=0.0)
    df.loc[idx,'GarageYrBlt'] = df.loc[idx,'YearBuilt']
    #The rest do not have garages so fill with 0, later convert to None 
    df['GarageYrBlt'] = df['GarageYrBlt'].fillna(value=0)
    
    #6. Impute 'Electrical' null values with the most common type 'SBrKr' -> motivated by the null value in test, is this data leakage?
    ### trainX['Electrical'].mode() = SBrkr
    df.loc[df['Electrical'].isna(),'Electrical'] = 'SBrkr'
    
    #7. JH:Specific additions: Replacing two values of GarageType to None
    df.loc[df['PID'] == 903426160,'GarageType'] = 'None'
    df.loc[df['PID'] == 910201180,'GarageType'] = 'None'

In [26]:
def convert_num_to_categorical(df):
    #Features that were originally numeric but should be treated as nominal categories since there is no clear 
    #advantage from applying a rank:
    num_to_nominal_cat_feats = [
                        'MSSubClass',
                        'KitchenAbvGr',
                        'BedroomAbvGr',
                        'MoSold',
                        'YrSold']
    for feat in num_to_nominal_cat_feats:
        df[feat] = df[feat].astype(str)
        
    #Add prefix to MSSubClass number code (string)
    df['MSSubClass'] = "MSSubClass_"+ df['MSSubClass']
    return num_to_nominal_cat_feats

## A3. Functions related to categorical features

In [27]:
#Only for checking that each categorical feature has been accounted for (assigned to nominal or ordinal)
#This function is not used in procesing 
def check_cat_feats(df):
    nominal_cat_feats = ['MSSubClass','MSZoning','Street','LotShape',
                     'LandContour','LotConfig','LandSlope','Neighborhood',
                     'Condition1','Condition2','BldgType','HouseStyle',
                     'RoofStyle','RoofMatl','Exterior1st','Exterior2nd',
                     'MasVnrType','Foundation','BsmtFinType1','BsmtFinType2',
                     'Heating','CentralAir','Electrical','Utilities',
                     'Functional','GarageType','MiscFeature','SaleType','SaleCondition']
    nominal_cat_feats += num_to_nominal_cat_feats #returned by convert_num_to_nomial_cat_feats
    
    ordinal_cat_feats = ['ExterQual','ExterCond','BsmtQual',
                     'BsmtCond','BsmtExposure','HeatingQC',
                     'KitchenQual','FireplaceQu','GarageFinish',
                     'GarageQual','GarageCond','PavedDrive',
                     'PoolQC','Fence','Alley']
    
    #check that all categorical features are assigned to either nominal_cat_feats or ordinal_cat_feats
    identify_feats(df)
    return len(set(cat_feats)-set(nominal_cat_feats)-set(ordinal_cat_feats))==0

In [28]:
#According to data dictionary, NA translates to 'None' (No access, No basement etc.) for the following categories:
def replace_na_with_none(df):
    na_means_none_cols = ['Alley','BsmtQual','BsmtCond','BsmtFinType1','BsmtFinType2',
                 'BsmtExposure','FireplaceQu','GarageType','GarageFinish',
                 'GarageQual','GarageCond','PoolQC','Fence','MiscFeature']
    for col in na_means_none_cols:
        df[col] = df[col].fillna(value = 'None')
        #print((df[col].unique()))    

In [29]:
def map_ordinal_cat(df):
    #Maps
    common_ranks_dict = {'None':0,'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}
    replace_map = {
        'ExterQual': common_ranks_dict,
        'ExterCond': common_ranks_dict,
        'BsmtQual': common_ranks_dict,
        'BsmtCond': common_ranks_dict,
        'BsmtExposure': {'None':0,'No':1,'Mn':2,'Av':3,'Gd':4}, 
        'HeatingQC': common_ranks_dict,
        'KitchenQual': common_ranks_dict,
        'FireplaceQu': common_ranks_dict,
        'GarageFinish': {'None':0,'Unf':1,'RFn':2,'Fin':3},
        'GarageQual': common_ranks_dict,
        'GarageCond': common_ranks_dict,
        'PavedDrive': {'N':0,'P':1,'Y':2},
        'PoolQC': {'None':0,'Fa':1,'TA':2,'Gd':3,'Ex':4},
        'Fence': {'None':0,'MnWw':1,'GdWo':2,'MnPrv':3,'GdPrv':4},
        'Alley': {'None':0,'Grvl':1,'Pave':2}
    }              
    #Replace strings with numbers 
    df.replace(replace_map, inplace=True)


# PART B: Overall Preprocessing Pipeline


In [30]:
### Load Data (temporary, can generalize)
data_train = pd.read_csv('/Users/nikiagrawal/Desktop/ML project/data/train.csv',index_col=0)
data_test = pd.read_csv('/Users/nikiagrawal/Desktop/ML project/data/test.csv',index_col=0)

# housing = pd.read_csv('Ames_Housing_Price_Data.csv')
# housing.drop('Unnamed: 0', axis=1, inplace=True)
# housing = housing[(housing['SaleCondition'] == 'Normal') | (housing['SaleCondition'] == 'Partial')].reset_index(drop=True)
# train, test = helper.stratified_split(housing,'Neighborhood')

In [31]:
trainX = data_train.drop('SalePrice',axis=1)
trainY = data_train['SalePrice']
testX = data_test.drop('SalePrice',axis=1)
testY = data_test['SalePrice']

# B1. Preprocess Training Data  

In [32]:
#Step 0. Inspect columns with missing data 
df = trainX
#Print out number catgeorical and numeric feats, and dataframe dimensions 
cat_feats,num_feats = identify_feats(df)
#Print out column, missing value count, missing value percentage 
print('\n')
cols_with_na(df)

categorical: 43
numerical: 37
df dims:(1871, 80)


LotFrontage   353   18.87
Alley   1749   93.48
MasVnrType   11   0.59
MasVnrArea   11   0.59
BsmtQual   54   2.89
BsmtCond   54   2.89
BsmtExposure   56   2.99
BsmtFinType1   54   2.89
BsmtFinType2   55   2.94
BsmtFullBath   1   0.05
BsmtHalfBath   1   0.05
FireplaceQu   877   46.87
GarageType   91   4.86
GarageYrBlt   92   4.92
GarageFinish   92   4.92
GarageQual   92   4.92
GarageCond   92   4.92
PoolQC   1866   99.73
Fence   1501   80.22
MiscFeature   1804   96.42
Total columns with NA values above threshold 0%: 20


In [33]:
#Step 1. Handle special case imputations
impute_missing_vals(df)
#Print out column, missing value count, missing value percentage 
cols_with_na(df)

Alley   1749   93.48
BsmtQual   54   2.89
BsmtCond   54   2.89
BsmtExposure   56   2.99
BsmtFinType1   54   2.89
BsmtFinType2   55   2.94
FireplaceQu   877   46.87
GarageType   91   4.86
GarageFinish   92   4.92
GarageQual   92   4.92
GarageCond   92   4.92
PoolQC   1866   99.73
Fence   1501   80.22
MiscFeature   1804   96.42
Total columns with NA values above threshold 0%: 14


In [34]:
#Handle remaining categorical imputations 
convert_num_to_categorical(df)
replace_na_with_none(df)
map_ordinal_cat(df)

#Print out number catgeorical and numeric feats, and dataframe dimensions 
cat_feats,num_feats = identify_feats(df)
#Print out column, missing value count, missing value percentage 
print('\n')
cols_with_na(df)

categorical: 33
numerical: 47
df dims:(1871, 80)


Total columns with NA values above threshold 0%: 0


# B2. Preprocess Test Data  


In [35]:
#Step 0. Inspect columns with missing data 
df = testX
#Print out number catgeorical and numeric feats, and dataframe dimensions 
cat_feats,num_feats = identify_feats(df)
#Print out column, missing value count, missing value percentage 
cols_with_na(df)

categorical: 43
numerical: 37
df dims:(624, 80)
LotFrontage   99   15.87
Alley   585   93.75
MasVnrType   3   0.48
MasVnrArea   3   0.48
BsmtQual   13   2.08
BsmtCond   13   2.08
BsmtExposure   13   2.08
BsmtFinType1   13   2.08
BsmtFinType2   13   2.08
Electrical   1   0.16
FireplaceQu   313   50.16
GarageType   25   4.01
GarageYrBlt   25   4.01
GarageFinish   25   4.01
GarageQual   25   4.01
GarageCond   25   4.01
PoolQC   620   99.36
Fence   492   78.85
MiscFeature   595   95.35
Total columns with NA values above threshold 0%: 19


In [36]:
#Step 1. Handle special case imputations
impute_missing_vals(df)
cols_with_na(df)

Alley   585   93.75
BsmtQual   13   2.08
BsmtCond   13   2.08
BsmtExposure   13   2.08
BsmtFinType1   13   2.08
BsmtFinType2   13   2.08
FireplaceQu   313   50.16
GarageType   25   4.01
GarageFinish   25   4.01
GarageQual   25   4.01
GarageCond   25   4.01
PoolQC   620   99.36
Fence   492   78.85
MiscFeature   595   95.35
Total columns with NA values above threshold 0%: 14


In [37]:
#Handle remaining categorical imputations 
convert_num_to_categorical(df)
replace_na_with_none(df)
map_ordinal_cat(df)

#Print out number catgeorical and numeric feats, and dataframe dimensions 
cat_feats,num_feats = identify_feats(df)
#Print out column, missing value count, missing value percentage 
print('\n')
cols_with_na(df)

categorical: 33
numerical: 47
df dims:(624, 80)


Total columns with NA values above threshold 0%: 0
